# Nonlinearsolve.jl Package

### CSCI 3656
### Michael Wallace, Kyle Okura, Alex Warren, Mark Worster

# Introduction

Nonlinearsolve.jl is a Julia package that serves as a unified interface for solving nonlinear systems and root-finding problems. It was created to give users the flexibility to switch between native algorithms and external libraries to find the fastest solver for their specific model. The package is capable of handling everything from small static problems to large-scale sparse systems, leveraging automatic differentiation and seamless integration with the Scientific Machine Learning (SciML) ecosystem for maximum efficiency.

# Setup

In [19]:
using Pkg
Pkg.add("NonlinearSolve")

Pkg.add("BenchmarkTools")

   Resolving package versions...
  No Changes to `C:\Users\kyleo\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\kyleo\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\kyleo\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\kyleo\.julia\environments\v1.11\Manifest.toml`


Possible topics to explore:

measure execution time between difference solver algorithms on the same or multiple problems
algorithms: NewtonRaphson, TrustRegion, Broyden, Klement
explore why some algorithms will be faster than others

visiualize and comprare the convergence speed to the runtime of each algorithm

explore tolerance within an algorithm
change the tolerance (absolute, relative, or both) and see how the runtime changes with precision
Show the tradeoff between speed and accuracy

explore how the difference algorithms perform when the initial guess is far from the true solution
for example newtons method can diverge if the inital guess is bad. Look into bracketing methods

show how the difference algorithms perform when working with large systems of equations


### Exploring the execution time between different rootfinding algorithms

In [20]:
using NonlinearSolve
using BenchmarkTools # For timing

#System of Equations
#F1 is a Circle, F2 is a line
function f_system!(F, u, p)
    x, y = u
    F[1] = x^2 + y^2 - 4
    F[2] = x - y - 1
end

# Initial guess
u0 = [1.0, 1.0]

prob = NonlinearProblem(f_system!, u0)

NonlinearProblem with uType Vector{Float64}. In-place: true
u0: 2-element Vector{Float64}:
 1.0
 1.0

In [21]:
using Printf #for better output formatting

"""
run_solver(prob, alg)
Solves the problem using a specific algorithm and returns the solution and the number of iterations
"""
function run_solver(prob, alg)
    sol = solve(prob, alg)
    return sol.u, sol.stats.nsteps
end


"""
race_algorithms(prob, alg_list)
Tests the problem on a list of algorithms
"""
function test_algorithms(prob, alg_list)
    println("----------------------------------------------------------------")
    @printf "%-30s | %-12s | %-15s\n" "Algorithm" "Iterations" "Time (Median)"
    println("----------------------------------------------------------------")

    for (name, alg) in alg_list
        # Get Iteration Count
        _, iterations = run_solver(prob, alg)

        # Benchmark Time
        time_sec = @belapsed solve($prob, $alg)

        # Convert to microseconds for better formatting
        time_microseconds = time_sec * 1e6

        @printf "%-30s | %-12d | %.2f μs\n" name iterations time_microseconds
    end
    println("----------------------------------------------------------------")
end

test_algorithms

In [22]:
methods = [
    ("NewtonRaphson", NewtonRaphson()), # The classic, requires Jacobian (usually fast/accurate)
    ("TrustRegion", TrustRegion()),   # Very stable, good if guess is bad
    ("Broyden", Broyden()),       # Quasi-Newton (doesn't compute full Jacobian)
    ("Klement", Klement())        # Another Quasi-Newton method
]

test_algorithms(prob, methods)

----------------------------------------------------------------
Algorithm                      | Iterations   | Time (Median)  
----------------------------------------------------------------
NewtonRaphson                  | 5            | 6.58 μs
TrustRegion                    | 7            | 8.20 μs
Broyden                        | 11           | 11.00 μs
Klement                        | 205          | 42.80 μs
----------------------------------------------------------------
